
We are analyzing user behaviour  on e-learning platform .Our task is to identify users who started a quiz but didn't submit with in 30 minutes- possible dropouts.



In [76]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,lead,unix_timestamp
from pyspark.sql.window import Window

In [77]:
spark=SparkSession.builder.appName("quiztest").getOrCreate()

In [78]:
data=[
 ("DEF1", "quiz_start", "2024-03-28 10:00:00"),
 ("DEF1", "quiz_submit", "2024-03-28 10:05:00"),
 ("DEF2", "quiz_start", "2024-03-28 11:00:00"),
 ("DEF2", "logout", "2024-03-28 11:20:00"),
 ("DEF3", "quiz_start", "2024-03-28 12:00:00"),
 ("DEF3", "quiz_submit", "2024-03-28 12:45:00"),
 ("DEF4", "quiz_start", "2024-03-28 13:00:00")
]

In [79]:
columns=["user_id","event_type","event_time"]

In [80]:
df=spark.createDataFrame(data,columns)
df.show()

+-------+-----------+-------------------+
|user_id| event_type|         event_time|
+-------+-----------+-------------------+
|   DEF1| quiz_start|2024-03-28 10:00:00|
|   DEF1|quiz_submit|2024-03-28 10:05:00|
|   DEF2| quiz_start|2024-03-28 11:00:00|
|   DEF2|     logout|2024-03-28 11:20:00|
|   DEF3| quiz_start|2024-03-28 12:00:00|
|   DEF3|quiz_submit|2024-03-28 12:45:00|
|   DEF4| quiz_start|2024-03-28 13:00:00|
+-------+-----------+-------------------+



In [81]:
df_res=df.withColumn("event_time",col("event_time").cast("timestamp"))
df_res.show()

+-------+-----------+-------------------+
|user_id| event_type|         event_time|
+-------+-----------+-------------------+
|   DEF1| quiz_start|2024-03-28 10:00:00|
|   DEF1|quiz_submit|2024-03-28 10:05:00|
|   DEF2| quiz_start|2024-03-28 11:00:00|
|   DEF2|     logout|2024-03-28 11:20:00|
|   DEF3| quiz_start|2024-03-28 12:00:00|
|   DEF3|quiz_submit|2024-03-28 12:45:00|
|   DEF4| quiz_start|2024-03-28 13:00:00|
+-------+-----------+-------------------+



In [82]:
filter_df=df_res.filter(col("event_type").isin("quiz_start","quiz_submit"))

In [83]:
window_spec=Window.partitionBy("user_id").orderBy("event_time")
window_spec

In [84]:
df_with_next=filter_df.withColumn("next_time",lead("event_type").over(window_spec)).withColumn("time_diff",unix_timestamp("next_time") - unix_timestamp("event_time"))

In [85]:
result=df_with_next.filter(
    (col("event_type")=="quiz_start") &
    ((col("next_time")!="quiz_submit")|(col("time_diff")>1800)|col("next_time").isNull())
).select(col("user_id"),col("event_time").alias("quiz_start_time"))

In [86]:
result.show()

+-------+-------------------+
|user_id|    quiz_start_time|
+-------+-------------------+
|   DEF2|2024-03-28 11:00:00|
|   DEF4|2024-03-28 13:00:00|
+-------+-------------------+

